### This notebook is a part of PGC ML project, identifying methylation signatures to predict PTSD in pre and post trauma. 

#### This first notebook contains code to pre-process the pre and post trauma  data from three Army cohorts

In [1]:
# First load the settings file
%run Settings.ipynb

In [2]:
# Function to read the data

def read_data(fname, dirpath = None, sheet_name = 0):
    
    """
    Function to load the data
    Parameters: 
    fname: file name including extension you want to read
    dirpath: path to the directory containing file, None by default
    sheet_name: Sheet name for reading excel sheets
    output: data frame
    
    """
    if dirpath is None:
        p = fname
    else:
        p = dirpath+fname
    
    if fname.endswith(".feather"):
        f = feather.read_feather(p)
    elif fname.endswith(".csv"):
        f = pd.read_csv(p)
    elif fname.endswith(".xlsx"):
        f = pd.read_excel(p, sheet_name = sheet_name)
        
    return(f)


def get_samples(df, cols):
    """
    Function to subset data
    Parameters:
    df: data frame
    cols: columns
    """
    meth = df.loc[:, df.columns.str.contains('|'.join(cols))]
    return(meth)

def get_trauma_exposed(df, col):
    """
    Function to get only trauman exposed samples
    Parameters: 
    df: data frame
    col: column name to filter the data frame
    """
    return(df[df[col] != 0])


def remove_duplicates(df, col):
    """
    Function to get only trauman exposed samples
    Parameters: 
    df: data frame
    col: column name that contain duplicate ids
    """
    return(df.drop_duplicates(subset= col))
    

#### MRS 

In [132]:
# load MRS
mrs = read_data(fname="G:/PGC ML/MRS/MRS_noob_qcd_crossReactiveProbesRemoved_combat_CP_wcovar_age_ptsd_allPreAsControls.feather")
mrs_pheno = read_data(fname="G:/PGC ML/MRS/MRS_Pheno_With_smoking_scores.csv")

In [133]:
# sort the rows 
mrs_pheno = mrs_pheno.sort_values(by =["studyid", "visit"],
                                ascending=True, axis=0)
mrs_pheno.shape

(254, 64)

In [134]:
mrs_pheno = get_trauma_exposed(df = mrs_pheno,
                              col = 'LECCUM_Stringent')
mrs_pheno.shape

(254, 64)

In [135]:
mrs_pheno['LECCUM_Stringent'].min()

1

In [136]:
# Check how many have two time points
mrs_pheno.groupby(["studyid"]).size().value_counts()

2    127
dtype: int64

In [137]:
# Childhoot trauma is recorded on the first visit, so copy that
# information for the second visit
mrs_pheno["CTQ_TOTAL"] = mrs_pheno.groupby(["studyid"])["CTQ_TOTAL"].ffill()

In [138]:
mrs_pheno[["studyid", "visit", "CTQ_TOTAL"]]

,studyid,visit,CTQ_TOTAL
77,1058,0.00000,NaN
76,1058,3.00000,NaN
63,1148,0.00000,NaN
62,1148,2.00000,NaN
245,1340,0.00000,NaN
244,1340,2.00000,NaN
107,2005,0.00000,NaN
106,2005,2.00000,NaN
64,2035,0.00000,NaN
65,2035,2.00000,NaN


In [139]:
# check shape
print("MRS beta shape :", mrs.shape)
print("MRS Pheno shape :", mrs_pheno.shape)
print("Unique study ids:", len(mrs_pheno['studyid'].unique()))

MRS beta shape : (821112, 255)
MRS Pheno shape : (254, 64)
Unique study ids: 127


In [140]:
# As MRS has pre and post, we will use only pre and post samples
def pull_pre_post(df, col, str_p):
    return df.loc[df[col].str.contains(str_p)]

In [141]:
# mrs_pheno_pre, mrs_pheno_post = [pull_pre_post(df = mrs_pheno, 
#                                                col = "ID", 
#                                                str_p = s) 
#                                  for s in ['PRE', 'POST']]

In [142]:
[x['Group'].value_counts() for x in [mrs_pheno]]
# print(mrs_pheno_post['Group'].value_counts())

[POST    127
 PRE     127
 Name: Group, dtype: int64]

In [143]:
# check if we have same ids in pre and post samples
# mrs_pheno_pre['studyid'].isin(mrs_pheno_post['studyid']).sum()

In [144]:
# Now get methylation samples that are in pheno for pre and post 
def col_list(df):
    return ["V1"] + df["BaseName"].tolist()
     
mrs_cols = col_list(df = mrs_pheno)        
mrs_meth = get_samples(df = mrs, cols=mrs_cols)

In [145]:
mrs_meth

,V1,201533580029_R01C01,201533580029_R02C01,201533580029_R03C01,201533580029_R04C01,201533580044_R01C01,201533580044_R02C01,201533580044_R05C01,201533580044_R06C01,201533580044_R07C01,201533580044_R08C01,201533590001_R01C01,201533590001_R02C01,201533590001_R03C01,201533590001_R04C01,201533590001_R05C01,201533590001_R06C01,201533590001_R07C01,201533590001_R08C01,201533590015_R01C01,201533590015_R02C01,201533590015_R03C01,201533590015_R04C01,201533590015_R05C01,201533590015_R06C01,201533590015_R07C01,201533590015_R08C01,201533590017_R01C01,201533590017_R02C01,201533590017_R03C01,201533590017_R04C01,201533590017_R05C01,201533590017_R06C01,201533590017_R07C01,201533590017_R08C01,201533590021_R01C01,201533590021_R02C01,201533590021_R03C01,201533590021_R04C01,201533590021_R05C01,201533590021_R06C01,201533590022_R01C01,201533590022_R02C01,201533590022_R03C01,201533590022_R04C01,201533590027_R01C01,201533590027_R02C01,201533590027_R03C01,201533590027_R04C01,201533590040_R01C01,201533590040_R02C01,201533590040_R03C01,201533590040_R04C01,201533590040_R05C01,201533590040_R06C01,201533590040_R07C01,201533590040_R08C01,201533590047_R01C01,201533590047_R02C01,201533590047_R03C01,201533590047_R04C01,201533590047_R07C01,201533590047_R08C01,201533590058_R01C01,201533590058_R02C01,201533590058_R03C01,201533590058_R04C01,201533590058_R05C01,201533590058_R06C01,201533590058_R07C01,201533590058_R08C01,201533590060_R01C01,201533590060_R02C01,201533590060_R05C01,201533590060_R06C01,201533590060_R07C01,201533590060_R08C01,201858500010_R03C01,201858500010_R04C01,201858500010_R05C01,201858500010_R06C01,201858500010_R07C01,201858500010_R08C01,201858500011_R01C01,201858500011_R02C01,201858500011_R03C01,201858500011_R04C01,201858500011_R05C01,201858500011_R06C01,201858500045_R01C01,201858500045_R02C01,201858500045_R03C01,201858500045_R04C01,201858500045_R05C01,201858500045_R06C01,201858500045_R07C01,201858500045_R08C01,201858500072_R01C01,201858500072_R02C01,201858500072_R03C01,...,201858500191_R03C01,201858500191_R04C01,201858500191_R05C01,201858500191_R06C01,201858500191_R07C01,201858500191_R08C01,201858500197_R03C01,201858500197_R04C01,201858500197_R05C01,201858500197_R06C01,201858500197_R07C01,201858500197_R08C01,201858500218_R01C01,201858500218_R02C01,201858500218_R03C01,201858500218_R04C01,201858500218_R05C01,201858500218_R06C01,201858500218_R07C01,201858500218_R08C01,201858500222_R01C01,201858500222_R02C01,201858500222_R05C01,201858500222_R06C01,201858500222_R07C01,201858500222_R08C01,201858500223_R01C01,201858500223_R02C01,201858500223_R03C01,201858500223_R04C01,201858500225_R03C01,201858500225_R04C01,201858500225_R05C01,201858500225_R06C01,201858500225_R07C01,201858500225_R08C01,201858500232_R01C01,201858500232_R02C01,201858500232_R03C01,201858500232_R04C01,201858500232_R05C01,201858500232_R06C01,201858500232_R07C01,201858500232_R08C01,201858500237_R01C01,201858500237_R02C01,201858500237_R03C01,201858500237_R04C01,201858500237_R05C01,201858500237_R06C01,201858500237_R07C01,201858500237_R08C01,201858500244_R01C01,201858500244_R02C01,201858500244_R05C01,201858500244_R06C01,201858500244_R07C01,201858500244_R08C01,201858500249_R05C01,201858500249_R06C01,201858500249_R07C01,201858500249_R08C01,201858500252_R01C01,201858500252_R02C01,201858500252_R03C01,201858500252_R04C01,201858500252_R05C01,201858500252_R06C01,201858500252_R07C01,201858500252_R08C01,201858500265_R01C01,201858500265_R02C01,201858500265_R03C01,201858500265_R04C01,201858500265_R05C01,201858500265_R06C01,201858500265_R07C01,201858500265_R08C01,201858500266_R01C01,201858500266_R02C01,201858500266_R03C01,201858500266_R04C01,201858500266_R05C01,201858500266_R06C01,201858500266_R07C01,201858500266_R08C01,202410280158_R01C01,202410280158_R02C01,202410280158_R03C01,202410280158_R04C01,202410280158_R07C01,202410280158_R08C01,202410280159_R01C01,202410280159_R02C01,202410280159_R03C01,202410280159_R04C01,202410280159_R05C01,202410280159_R06C01,202410280159_R07C01,202410280159_R08C01
0,cg18478105

In [146]:
# Lets check if we have all the samples we were supposed to have
# from pre and post samples
print("All pre matching:", mrs_pheno['BaseName'].isin(mrs_meth.columns).all())

All pre matching: True


### ArmySTARRS

In [147]:
army_path = "G:/PGC ML/ArmySTARRS/"
army_pheno = read_data(fname="pre_post_armystarrs_Pheno_ML.csv", dirpath=army_path)
army_meth = read_data(fname="Starrs_noob_qcd_crossReactiveProbesRemoved_combat_CP_wcovar_age2TP_ptsd_allPreAsControls.feather",
                     dirpath=army_path)

In [148]:
print("Armystarrs beta shape :", army_meth.shape)
print("Armystarrs Pheno shape :", army_pheno.shape)

Armystarrs beta shape : (821329, 438)
Armystarrs Pheno shape : (430, 80)


In [149]:
# samples for two visits
army_pheno["visit"].value_counts()

0    215
2    215
Name: visit, dtype: int64

In [150]:
# In armystarrs we have different trauma variables
# lets conside either non-deployment related trauma or 
# deployment related trauma
army_pheno["trauma_exposed_critA"].value_counts()

1    346
0     84
Name: trauma_exposed_critA, dtype: int64

In [151]:
# lets keep only trauma exposed
army_pheno = get_trauma_exposed(df = army_pheno, 
                               col = 'trauma_exposed_critA')

In [152]:
army_pheno['trauma_exposed_critA'].min()

1

In [153]:
army_pheno['visit'].value_counts()

0    173
2    173
Name: visit, dtype: int64

In [154]:
# get only visit 2
# army_v0_pheno, army_v2_pheno = [army_pheno[army_pheno['visit'] == i]
#                                 for i in [0,2]]

In [155]:
[x['visit'].value_counts() for x in [army_pheno]]

[0    173
 2    173
 Name: visit, dtype: int64]

In [156]:
[x["trauma_exposed_critA"].value_counts() 
 for x in [army_pheno]]

[1    346
 Name: trauma_exposed_critA, dtype: int64]

In [157]:
# drop columns that have all nas
army_pheno = army_pheno.dropna(axis=1, how = 'all')

In [158]:
[x.shape for x in [army_pheno]]

[(346, 75)]

In [159]:
print("No of unique ppts :", len(army_pheno['EWAS_id_new'].unique()))

No of unique ppts : 173


In [160]:
[x.isna().sum() for x in [army_pheno]]

[BaseName                            0
 EWAS_id                             0
 EWAS_id_new                         0
 GWAS_id                             0
 visitkey                            0
 visit                               0
 CURRENT_PTSD                       13
 LIFETIME_PTSD                       0
 TOBACCOUSE                          0
 P30DFREQ                           13
 MaltreatmentGlobal                  0
 nondeploy_trauma_exposed_critA      0
 deploy_trauma_exposed_critA         0
 trauma_exposed_critA                0
 pd_s                                0
 AGE                                13
 genetic_ancestry                    0
 hisp                                2
 race                                2
 PCL6_t0                           175
 PCL17_t23                         189
 pcl6_b_2q                         175
 pcl6_c_2q                         175
 pcl6_d_2q                         175
 pcl17_b_5q                        190
 pcl17_c_7q              

In [161]:
# Now get methylation samples that are in pheno
army_cols  = col_list(df=army_pheno)
army_meth  = get_samples(df = army_meth, cols = army_cols)
                            

In [162]:
[x.shape for x in [army_meth]]

[(821329, 347)]

In [163]:
# army_v0_pheno.columns.isin(army_v2_pheno.columns)

In [164]:
# ArmySTARRS has two age columns, let drop one
army_pheno.drop('age', axis = 1, inplace=True)

In [165]:
army_pheno.columns

Index(['BaseName', 'EWAS_id', 'EWAS_id_new', 'GWAS_id', 'visitkey', 'visit',
       'CURRENT_PTSD', 'LIFETIME_PTSD', 'TOBACCOUSE', 'P30DFREQ',
       'MaltreatmentGlobal', 'nondeploy_trauma_exposed_critA',
       'deploy_trauma_exposed_critA', 'trauma_exposed_critA', 'pd_s', 'AGE',
       'genetic_ancestry', 'hisp', 'race', 'PCL6_t0', 'PCL17_t23', 'pcl6_b_2q',
       'pcl6_c_2q', 'pcl6_d_2q', 'pcl17_b_5q', 'pcl17_c_7q', 'pcl17_d_5q',
       'GWAS', 'n', 'specimen_date', 'specimen_age', 'specimen_type', 'array',
       'methylationid', 'sentrix_id', 'sentrix_position', 'data_sharing',
       'case_status', 'PTSD_measure', 'PTSD_symptoms_pcl6',
       'PTSD_symptoms_pcl617', 'pheno_date', 's', 'Study', 'SampleID',
       'smoking', 'ptsd_case', 'bestpop_oneweek', 'Gender', 'CD8T.EPIC',
       'CD4T.EPIC', 'NK.EPIC', 'Bcell.EPIC', 'Mono.EPIC', 'Neu.EPIC',
       'CD8T.EPICnoob', 'CD4T.EPICnoob', 'NK.EPICnoob', 'Bcell.EPICnoob',
       'Mono.EPICnoob', 'Neu.EPICnoob', 'Comp.1', 'Comp.2', '

### PRISMO

In [166]:
prismo_path ="G:/PGC ML/PRISMO/"
prismo_pheno = read_data(fname="pre_post_prismo_Pheno_ML.csv", dirpath=prismo_path)
prismo_meth = read_data(fname="Prismo_noob_qcd_crossReactiveProbesRemoved_combat_CP_wcovar_age_ptsd_allPreAsControls.feather",
                     dirpath=prismo_path)

In [167]:
print("Prismo beta shape :", prismo_meth.shape)
print("Prismo Pheno shape :", prismo_pheno.shape)

Prismo beta shape : (821031, 237)
Prismo Pheno shape : (118, 64)


In [168]:
prismo_pheno["visit"].value_counts()


0_epic    59
2_epic    59
Name: visit, dtype: int64

In [169]:
prismo_pheno = get_trauma_exposed(df = prismo_pheno,
                                 col = 'Pes_number')
prismo_pheno.shape

(117, 64)

In [170]:
# prismo_pheno, prismo_v2_pheno = [prismo_pheno[prismo_pheno["visit"] == x]
#                                     for x in ["0_epic", "2_epic"]]

In [171]:
[x.shape for x in [prismo_pheno]]

[(117, 64)]

In [172]:
prismo_pheno.shape

(117, 64)

In [173]:
print("No of unique ppts pre :", len(prismo_pheno['EWAS_id'].unique()))

No of unique ppts pre : 59


In [174]:
prismo_pheno.isna().sum()

BaseName                    0
EWAS_id                     0
visit                       0
visitkey                    0
CURRENT_PTSD               12
SMOKING                    73
smoking_status              0
ETItot                      4
ETIalg                      2
ETIlich                     2
ETIgeest                    4
ETIs                        4
Pes_number                 67
gender                      0
AGE                         0
ancestry                    0
TOTAL_SCORE                12
REEXPERIENCE               12
AVOID                      12
HYPERAROUSAL               12
DEPRESSION                  8
ANXIETY                     8
GWAS                        0
n                           0
GWAS_id                     0
specimen_date              10
specimen_age               10
specimen_type               0
array                       0
methylation_id              0
sentrix_id                  0
sentrix_position            0
data_sharing                0
case_statu

In [175]:
prismo_cols = col_list(df = prismo_pheno)
        
prismo_meth = get_samples(df = prismo_meth, cols = prismo_cols)

In [176]:
[x.shape for x in [prismo_meth]]

[(821031, 118)]

In [177]:
prismo_pheno.columns

Index(['BaseName', 'EWAS_id', 'visit', 'visitkey', 'CURRENT_PTSD', 'SMOKING',
       'smoking_status', 'ETItot', 'ETIalg', 'ETIlich', 'ETIgeest', 'ETIs',
       'Pes_number', 'gender', 'AGE', 'ancestry', 'TOTAL_SCORE',
       'REEXPERIENCE', 'AVOID', 'HYPERAROUSAL', 'DEPRESSION', 'ANXIETY',
       'GWAS', 'n', 'GWAS_id', 'specimen_date', 'specimen_age',
       'specimen_type', 'array', 'methylation_id', 'sentrix_id',
       'sentrix_position', 'data_sharing', 'case_status', 'PTSD_measure',
       'PTSD_symptoms', 'pheno_date', 's', 'age', 'genetic_ancestry', 'Study',
       'SameIDs', 'Group', 'time', 'ptsdActualCont', 'ptsdActual', 'Comp.1',
       'Comp.2', 'Comp.3', 'Comp.4', 'Comp.5', 'Comp.6', 'Comp.7', 'Comp.8',
       'Comp.9', 'Comp.10', 'CD8T.Epic', 'CD4T.Epic', 'NK.Epic', 'Bcell.Epic',
       'Mono.Epic', 'Neu.Epic', 'ptsdActualCont_01scaled', 'SmoS'],
      dtype='object')

In [178]:
# just a thought ----------------
# in DNHS we have Remitted samples as well
# So when we use ptsdpm, we need to remove those remitted ones


In [179]:
# dnhs_pheno.iloc[:5, :5]

In [180]:
# gtp.iloc[:5, :5]

#### Now combine all data

In [181]:
def rename_column(df, col_indx):
    return df.rename(columns = {df.columns[col_indx]: 'CpGs'})

# Make a list of dfs 
all_meth_dfs = [mrs_meth, army_meth, prismo_meth]

# rename the first column
all_meth_dfs = [rename_column(df = x, col_indx = 0)
                for x in all_meth_dfs]


In [182]:
[x.iloc[:5, :5] for x in all_meth_dfs]

[         CpGs  201533580029_R01C01  201533580029_R02C01  201533580029_R03C01  \
 0  cg18478105              0.01471              0.01638              0.01675   
 1  cg09835024              0.02721              0.03539              0.02973   
 2  cg14361672              0.87702              0.90871              0.88785   
 3  cg01763666              0.80343              0.82561              0.82502   
 4  cg12950382              0.90471              0.91791              0.88500   
 
    201533580029_R04C01  
 0              0.01356  
 1              0.03160  
 2              0.86194  
 3              0.88384  
 4              0.92514  ,
          CpGs  201858500071_R03C01  201858500071_R04C01  201858500071_R07C01  \
 0  cg18478105              0.01526              0.01589              0.01544   
 1  cg09835024              0.02771              0.02711              0.02504   
 2  cg14361672              0.88314              0.92609              0.92533   
 3  cg01763666              0.8

In [183]:
# Combine all methylation data 
from functools import reduce
dfs_merged  = reduce(lambda left, 
                     right: pd.merge(left, 
                                     right,
                                     on = "CpGs",
                                     how='inner'), 
                     all_meth_dfs) 

In [184]:
[x.shape for x in [dfs_merged]]

[(820498, 718)]

In [185]:
[len(x['CpGs'].unique()) for x in [dfs_merged]]

[820498]

In [186]:
dfs_merged.iloc[:5, :5]

,CpGs,201533580029_R01C01,201533580029_R02C01,201533580029_R03C01,201533580029_R04C01
0,cg18478105,0.01471,0.01638,0.01675,0.01356
1,cg09835024,0.02721,0.03539,0.02973,0.03160
2,cg14361672,0.87702,0.90871,0.88785,0.86194
3,cg01763666,0.80343,0.82561,0.82502,0.88384
4,cg12950382,0.90471,0.91791,0.88500,0.92514


#### We also need to combine the phenotypes, but before we do that we need to get the common variables

In [187]:
# dnhs_pheno.columns

In [188]:
# gtp_pheno_comb.columns

In [189]:
mrs_pheno.columns

Index(['BaseName', 'EWAS_id', 'GWAS_id', 'studyid', 'visitkey', 'visit',
       'specimen_date', 'specimen_type', 'array', 'sentrix_id',
       'sentrix_position', 'data_sharing', 'case_status', 'PTSD_measure',
       'PTSD_symptoms', 'pheno_date', 'CAPSF1I2s', 'TOBAC_lifetime_user',
       'current_smoking', 'current_snuff', 'current_chew', 'current_any',
       'smokelast', 'nicotine_use', 'CTQ_TOTAL', 'LECCUM_Stringent', 'Sex',
       'Age', 'genetic_ancestry', 'ba_ethnic', 'ba_race', 'CAPStots', 'CAPSBs',
       'CAPSCs', 'CAPSDs', 'BDI2_mod_sev', 'BDI2_SUM', 'BAI_mod_sev',
       'BAI_SUM', 'GWAS', 'Study', 'ID', 'Group', 'Case', 'PCL_SUM', 'tobacco',
       'post_deployment_visit', 'age', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5',
       'CD8T', 'CD4T', 'NK', 'Bcell', 'Mono', 'Neu', 'Comp.1', 'Comp.2',
       'Comp.3', 'PCL_SUM_01scaled', 'SmoS'],
      dtype='object')

In [190]:
army_pheno.columns

Index(['BaseName', 'EWAS_id', 'EWAS_id_new', 'GWAS_id', 'visitkey', 'visit',
       'CURRENT_PTSD', 'LIFETIME_PTSD', 'TOBACCOUSE', 'P30DFREQ',
       'MaltreatmentGlobal', 'nondeploy_trauma_exposed_critA',
       'deploy_trauma_exposed_critA', 'trauma_exposed_critA', 'pd_s', 'AGE',
       'genetic_ancestry', 'hisp', 'race', 'PCL6_t0', 'PCL17_t23', 'pcl6_b_2q',
       'pcl6_c_2q', 'pcl6_d_2q', 'pcl17_b_5q', 'pcl17_c_7q', 'pcl17_d_5q',
       'GWAS', 'n', 'specimen_date', 'specimen_age', 'specimen_type', 'array',
       'methylationid', 'sentrix_id', 'sentrix_position', 'data_sharing',
       'case_status', 'PTSD_measure', 'PTSD_symptoms_pcl6',
       'PTSD_symptoms_pcl617', 'pheno_date', 's', 'Study', 'SampleID',
       'smoking', 'ptsd_case', 'bestpop_oneweek', 'Gender', 'CD8T.EPIC',
       'CD4T.EPIC', 'NK.EPIC', 'Bcell.EPIC', 'Mono.EPIC', 'Neu.EPIC',
       'CD8T.EPICnoob', 'CD4T.EPICnoob', 'NK.EPICnoob', 'Bcell.EPICnoob',
       'Mono.EPICnoob', 'Neu.EPICnoob', 'Comp.1', 'Comp.2', '

In [191]:
prismo_pheno.columns

Index(['BaseName', 'EWAS_id', 'visit', 'visitkey', 'CURRENT_PTSD', 'SMOKING',
       'smoking_status', 'ETItot', 'ETIalg', 'ETIlich', 'ETIgeest', 'ETIs',
       'Pes_number', 'gender', 'AGE', 'ancestry', 'TOTAL_SCORE',
       'REEXPERIENCE', 'AVOID', 'HYPERAROUSAL', 'DEPRESSION', 'ANXIETY',
       'GWAS', 'n', 'GWAS_id', 'specimen_date', 'specimen_age',
       'specimen_type', 'array', 'methylation_id', 'sentrix_id',
       'sentrix_position', 'data_sharing', 'case_status', 'PTSD_measure',
       'PTSD_symptoms', 'pheno_date', 's', 'age', 'genetic_ancestry', 'Study',
       'SameIDs', 'Group', 'time', 'ptsdActualCont', 'ptsdActual', 'Comp.1',
       'Comp.2', 'Comp.3', 'Comp.4', 'Comp.5', 'Comp.6', 'Comp.7', 'Comp.8',
       'Comp.9', 'Comp.10', 'CD8T.Epic', 'CD4T.Epic', 'NK.Epic', 'Bcell.Epic',
       'Mono.Epic', 'Neu.Epic', 'ptsdActualCont_01scaled', 'SmoS'],
      dtype='object')

In [192]:
# raname the first columns
# DNHS
# dnhs_pheno = dnhs_pheno.rename(columns={'X':'BaseName', 'race6cat':'Race',
#                           'childhood_cum_trauma': 'Childhood_MT',
#                            'life_worst_intrusion': 'Intrusion',
#                            'life_worst_avoidance': 'Avoidance',
#                            'life_worst_hyperarousal': 'Hyperarousal',
#                            'phq9sum': 'MDD',
#                            'gad7sum': 'GAD',
#                            'Life_PTS_severity': 'PTS_severity'
#                           })


In [193]:
# GTP
# gtp_pheno_comb = gtp_pheno_comb.rename(columns={'Unnamed: 0':'BaseName',
#                                'mergedcapsandpsswinthin30days':'PTSDpm', 
#                                'Life_PTSD_01': 'PTSDLife',
#                                'age_x': 'Age',
#                                'tei_total_types_experienced_somewitness':'TraumaNum',
#                               'caps_life_freqplusintens_combined': 'PTS_severity',
#                                'PSS_Intrusive': 'Intrusion',
#                                'PSS_avoidnumb': 'Avoidance',
#                                'PSS_hyperarousal': 'Hyperarousal',
#                                'BDItotalscore': 'MDD',
#                                'CTQTOT': 'Childhood_MT',
#                                'pc1': 'Comp.1',
#                                'pc2': 'Comp.2',
#                                'pc3': 'Comp.3',
                               
#                               })

In [194]:
# MRS
mrs_pheno = mrs_pheno.rename(columns={'ba_race': 'race',
                               'CAPSF1I2s': 'PTSDpm',
                               'Lifetime.PTSD' : 'PTSDLife',
                               'LECCUM_Stringent': 'TraumaNum',
                               'CAPStots': 'PTS_severity',
                               'CAPSBs': 'Intrusion',
                               'CAPSCs': 'Avoidance',
                               'CAPSDs': 'Hyperarousal',
                               'BDI2_SUM': 'MDD',
                               'BAI_mod_sev': 'GAD',
                               'CTQ_TOTAL' : 'Childhood_MT',
                               'Sex': 'Gender'
                              })

In [195]:
# raname the first columns
# ArmyStarrs
army_pheno = army_pheno.rename(columns={'race':'Race',
                        'CURRENT_PTSD': "PTSDpm",
                        'LIFETIME_PTSD': 'PTSDLife',
                        'MaltreatmentGlobal': 'Childhood_MT',
                        'trauma_exposed_critA': 'TraumaNum',
                        'pcl17_b_5q': 'Intrusion',
                        'pcl17_c_7q': 'Avoidance',
                        'pcl17_d_5q': 'Hyperarousal',
                        'PCL17_t23': 'PTS_severity',
                        'CD8T.EPICnoob': 'CD8T',
                        'CD4T.EPICnoob': 'CD4T',
                        'NK.EPICnoob': 'NK',
                        'Bcell.EPICnoob': 'Bcell',
                        'Mono.EPICnoob': 'Mono',
                        'Neu.EPICnoob': 'Neu'
                          })

In [196]:
# Prismo
prismo_pheno  = prismo_pheno.rename(columns={'ancestry':'Race',
                          'CURRENT_PTSD': "PTSDpm",
                          'LIFETIME_PTSD': 'PTSDLife',
                          'ETItot': 'Childhood_MT',
                          'Pes_number': 'TraumaNum',
                          'REEXPERIENCE': 'Intrusion',
                          'AVOID': 'Avoidance',
                          'HYPERAROUSAL': 'Hyperarousal',
                          'TOTAL_SCORE': 'PTS_severity',
                          'CD8T.Epic': 'CD8T',
                          'CD4T.Epic': 'CD4T',
                          'NK.Epic': 'NK',
                          'Bcell.Epic': 'Bcell',
                          'Mono.Epic': 'Mono',
                          'Neu.Epic': 'Neu',
                          'gender': 'Gender'
                          })

In [197]:
# dnhs_pheno.columns

In [198]:
# gtp_pheno_comb.columns

In [199]:
# For mrs, both Pcs from gwas and methylation data are available
# Comp.1, Comp.2, Comp.3 are methylation
[x.columns for x in [mrs_pheno]]

[Index(['BaseName', 'EWAS_id', 'GWAS_id', 'studyid', 'visitkey', 'visit',
        'specimen_date', 'specimen_type', 'array', 'sentrix_id',
        'sentrix_position', 'data_sharing', 'case_status', 'PTSD_measure',
        'PTSD_symptoms', 'pheno_date', 'PTSDpm', 'TOBAC_lifetime_user',
        'current_smoking', 'current_snuff', 'current_chew', 'current_any',
        'smokelast', 'nicotine_use', 'Childhood_MT', 'TraumaNum', 'Gender',
        'Age', 'genetic_ancestry', 'ba_ethnic', 'race', 'PTS_severity',
        'Intrusion', 'Avoidance', 'Hyperarousal', 'BDI2_mod_sev', 'MDD', 'GAD',
        'BAI_SUM', 'GWAS', 'Study', 'ID', 'Group', 'Case', 'PCL_SUM', 'tobacco',
        'post_deployment_visit', 'age', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5',
        'CD8T', 'CD4T', 'NK', 'Bcell', 'Mono', 'Neu', 'Comp.1', 'Comp.2',
        'Comp.3', 'PCL_SUM_01scaled', 'SmoS'],
       dtype='object')]

In [200]:
[x.columns for x in[army_pheno]]

[Index(['BaseName', 'EWAS_id', 'EWAS_id_new', 'GWAS_id', 'visitkey', 'visit',
        'PTSDpm', 'PTSDLife', 'TOBACCOUSE', 'P30DFREQ', 'Childhood_MT',
        'nondeploy_trauma_exposed_critA', 'deploy_trauma_exposed_critA',
        'TraumaNum', 'pd_s', 'AGE', 'genetic_ancestry', 'hisp', 'Race',
        'PCL6_t0', 'PTS_severity', 'pcl6_b_2q', 'pcl6_c_2q', 'pcl6_d_2q',
        'Intrusion', 'Avoidance', 'Hyperarousal', 'GWAS', 'n', 'specimen_date',
        'specimen_age', 'specimen_type', 'array', 'methylationid', 'sentrix_id',
        'sentrix_position', 'data_sharing', 'case_status', 'PTSD_measure',
        'PTSD_symptoms_pcl6', 'PTSD_symptoms_pcl617', 'pheno_date', 's',
        'Study', 'SampleID', 'smoking', 'ptsd_case', 'bestpop_oneweek',
        'Gender', 'CD8T.EPIC', 'CD4T.EPIC', 'NK.EPIC', 'Bcell.EPIC',
        'Mono.EPIC', 'Neu.EPIC', 'CD8T', 'CD4T', 'NK', 'Bcell', 'Mono', 'Neu',
        'Comp.1', 'Comp.2', 'Comp.3', 'Comp.4', 'Comp.5', 'Comp.6', 'Comp.7',
        'Comp.8', 'Comp.

In [201]:
[x.columns for x in [prismo_pheno]]

[Index(['BaseName', 'EWAS_id', 'visit', 'visitkey', 'PTSDpm', 'SMOKING',
        'smoking_status', 'Childhood_MT', 'ETIalg', 'ETIlich', 'ETIgeest',
        'ETIs', 'TraumaNum', 'Gender', 'AGE', 'Race', 'PTS_severity',
        'Intrusion', 'Avoidance', 'Hyperarousal', 'DEPRESSION', 'ANXIETY',
        'GWAS', 'n', 'GWAS_id', 'specimen_date', 'specimen_age',
        'specimen_type', 'array', 'methylation_id', 'sentrix_id',
        'sentrix_position', 'data_sharing', 'case_status', 'PTSD_measure',
        'PTSD_symptoms', 'pheno_date', 's', 'age', 'genetic_ancestry', 'Study',
        'SameIDs', 'Group', 'time', 'ptsdActualCont', 'ptsdActual', 'Comp.1',
        'Comp.2', 'Comp.3', 'Comp.4', 'Comp.5', 'Comp.6', 'Comp.7', 'Comp.8',
        'Comp.9', 'Comp.10', 'CD8T', 'CD4T', 'NK', 'Bcell', 'Mono', 'Neu',
        'ptsdActualCont_01scaled', 'SmoS'],
       dtype='object')]

In [202]:
# We have two age columns, lets drop one
mrs_pheno.drop(columns=['Age'], inplace=True)

In [203]:
[x.drop(columns=['AGE'], inplace=True) 
 for x in [army_pheno, prismo_pheno]]

[None, None]

In [204]:
need_cols = ['BaseName', '^EWAS_id$', '^visit$', '^visitkey$',
             'Gender','race$', '^Age$', 
             'PTSDpm', 'PTSDLife', 'TraumaNum', 
             'CD8T$', 'CD4T$', 'NK$', 'Bcell$', 'Mono$',
             'Neu$','PTS_severity', 'Childhood_MT',
             'Intrusion', 'Avoidance', 'Hyperarousal', '^MDD$',
            'Comp.2', 'Comp.3', 'Study$', 'SmoS']

In [205]:
import re
def get_cols(df, cols, case=None, title=None, sort=None):
    """
    Function to get required columns
    Parameters: 
    df: data frame
    cols: columns that need to be fetched
    case: If case should be ignored,  None by default
    
    Output: The dataframe with selected columns
    """
    if case is True:
        d = df.filter(regex=re.compile('|'.join(cols), re.IGNORECASE))
    else:
        d = df.filter(regex= re.compile('|'.join(cols)))
        
    if title is True:
        d.columns = [i.title() for i in d.columns]
        
    if sort is True:
        d = d.sort_index(axis=1)
        
    return(d)


# get the frequency of elements
def get_frequency(df, col):
    return(df[col].value_counts())


In [206]:
# now get the required columns from all dfs

all_phenos = [mrs_pheno, army_pheno, prismo_pheno]

phenos_sub = [get_cols(df = x, cols=need_cols, case=True,
                      title=True, sort=True) for x in all_phenos]

cohorts = ["mrs", "armystarrs", "prismo"]

phenos_sub = dict(zip(cohorts, phenos_sub)) # make a dictionary

In [207]:
phenos_sub.keys()

dict_keys(['mrs', 'armystarrs', 'prismo'])

In [208]:
[x.columns for x in phenos_sub.values()]

[Index(['Age', 'Avoidance', 'Basename', 'Bcell', 'Cd4T', 'Cd8T', 'Childhood_Mt',
        'Comp.2', 'Comp.3', 'Ewas_Id', 'Gender', 'Hyperarousal', 'Intrusion',
        'Mdd', 'Mono', 'Neu', 'Nk', 'Pts_Severity', 'Ptsdpm', 'Race', 'Smos',
        'Study', 'Traumanum', 'Visit', 'Visitkey'],
       dtype='object'),
 Index(['Age', 'Avoidance', 'Basename', 'Bcell', 'Cd4T', 'Cd8T', 'Childhood_Mt',
        'Comp.2', 'Comp.3', 'Ewas_Id', 'Gender', 'Hyperarousal', 'Intrusion',
        'Mono', 'Neu', 'Nk', 'Pts_Severity', 'Ptsdlife', 'Ptsdpm', 'Race',
        'Smos', 'Study', 'Traumanum', 'Visit', 'Visitkey'],
       dtype='object'),
 Index(['Age', 'Avoidance', 'Basename', 'Bcell', 'Cd4T', 'Cd8T', 'Childhood_Mt',
        'Comp.2', 'Comp.3', 'Ewas_Id', 'Gender', 'Hyperarousal', 'Intrusion',
        'Mono', 'Neu', 'Nk', 'Pts_Severity', 'Ptsdpm', 'Race', 'Smos', 'Study',
        'Traumanum', 'Visit', 'Visitkey'],
       dtype='object')]

In [209]:
# get columns of each df
mrs_cols, army_cols, prismo_cols = [x.columns for x in phenos_sub.values()]

In [210]:
mrs_cols

Index(['Age', 'Avoidance', 'Basename', 'Bcell', 'Cd4T', 'Cd8T', 'Childhood_Mt',
       'Comp.2', 'Comp.3', 'Ewas_Id', 'Gender', 'Hyperarousal', 'Intrusion',
       'Mdd', 'Mono', 'Neu', 'Nk', 'Pts_Severity', 'Ptsdpm', 'Race', 'Smos',
       'Study', 'Traumanum', 'Visit', 'Visitkey'],
      dtype='object')

In [211]:
army_cols

Index(['Age', 'Avoidance', 'Basename', 'Bcell', 'Cd4T', 'Cd8T', 'Childhood_Mt',
       'Comp.2', 'Comp.3', 'Ewas_Id', 'Gender', 'Hyperarousal', 'Intrusion',
       'Mono', 'Neu', 'Nk', 'Pts_Severity', 'Ptsdlife', 'Ptsdpm', 'Race',
       'Smos', 'Study', 'Traumanum', 'Visit', 'Visitkey'],
      dtype='object')

In [212]:
prismo_cols

Index(['Age', 'Avoidance', 'Basename', 'Bcell', 'Cd4T', 'Cd8T', 'Childhood_Mt',
       'Comp.2', 'Comp.3', 'Ewas_Id', 'Gender', 'Hyperarousal', 'Intrusion',
       'Mono', 'Neu', 'Nk', 'Pts_Severity', 'Ptsdpm', 'Race', 'Smos', 'Study',
       'Traumanum', 'Visit', 'Visitkey'],
      dtype='object')

In [213]:
# check if column names are matching
# (dnhs_cols == gtp_cols).all()

In [214]:
import warnings
def matching(l1, l2):
    """
    Function to compare two lists and check the order
    
    Parameters:
    l1: list 1
    l2: list 2 
    """
    
    print("Total elements in l1 :", len(l1))
    print("Total elements in l2 :", len(l2))
    m = len([l for l in l1 if l in l2])
    print("Elements matching between l1 and l2 :", m)
    if(len(l1) == len(l2)):
        print("All in order :", (l1 == l2).all())
    else:
        elm = list(set(l1).difference(l2))
        print(elm)
        l1 = [x for x in l1 if x not in elm]
        print(l1)
        print("All common elements in order :", (l1 == l2).all())
        

In [215]:
matching(l1 = mrs_cols, l2 = army_cols)

Total elements in l1 : 25
Total elements in l2 : 25
Elements matching between l1 and l2 : 24
All in order : False


In [216]:
matching(l1 = army_cols, l2 = prismo_cols)

Total elements in l1 : 25
Total elements in l2 : 24
Elements matching between l1 and l2 : 24
['Ptsdlife']
['Age', 'Avoidance', 'Basename', 'Bcell', 'Cd4T', 'Cd8T', 'Childhood_Mt', 'Comp.2', 'Comp.3', 'Ewas_Id', 'Gender', 'Hyperarousal', 'Intrusion', 'Mono', 'Neu', 'Nk', 'Pts_Severity', 'Ptsdpm', 'Race', 'Smos', 'Study', 'Traumanum', 'Visit', 'Visitkey']
All common elements in order : True


In [217]:
# matching(l1 = dnhs_cols, l2 = army_cols)

In [218]:
# matching(l1 = dnhs_cols, l2 = prismo_cols)

In [219]:
# common in all
list(set(mrs_cols) & set(army_cols) & set(prismo_cols))

['Cd8T',
 'Gender',
 'Comp.3',
 'Pts_Severity',
 'Nk',
 'Ewas_Id',
 'Intrusion',
 'Mono',
 'Age',
 'Study',
 'Visitkey',
 'Cd4T',
 'Hyperarousal',
 'Race',
 'Ptsdpm',
 'Avoidance',
 'Traumanum',
 'Neu',
 'Basename',
 'Visit',
 'Childhood_Mt',
 'Smos',
 'Comp.2',
 'Bcell']

In [220]:
[x.iloc[:5, :5] for x in phenos_sub.values()]

[     Age  Avoidance             Basename   Bcell    Cd4T
 77    20          3  201858500010_R04C01 0.06982 0.11557
 76    21          8  201858500010_R03C01 0.03482 0.08983
 63    21         12  201533590058_R02C01 0.02957 0.12690
 62    22          3  201533590058_R01C01 0.02668 0.15131
 245   21          4  202410280158_R08C01 0.00214 0.12064,
     Age  Avoidance             Basename   Bcell    Cd4T
 2    30        NaN  201858500071_R03C01 0.06154 0.10474
 3    32        NaN  201858500071_R04C01 0.04358 0.13446
 6    29        NaN  201858500071_R07C01 0.03540 0.09427
 7    30    7.00000  201858500071_R08C01 0.04971 0.16086
 10   35   22.00000  201858500082_R03C01 0.04404 0.14152,
        Age  Avoidance             Basename   Bcell    Cd4T
 0 25.20128   16.00000  201228780091_R03C01 0.05651 0.14886
 1 25.64206   16.00000  201228780091_R04C01 0.04168 0.12357
 2 26.34908    9.00000  201228780091_R07C01 0.03384 0.09820
 3 26.84189   18.00000  201228780091_R08C01 0.04501 0.12114
 4 21.22

In [221]:
# phenos_sub['gtp']

In [222]:
# convert to int
# gtp_p_sub = phenos_sub['gtp'].astype({"Traumanum":'int',
#                              })
# get_frequency(df=gtp_p_sub, col='Ptsdlife')

In [223]:
# nas in ptsd life
# gtp_p_sub['Ptsdlife'].isna().sum()

In [224]:
# phenos_sub['dnhs']

In [225]:
# get number of males and females
# all are males in MRS, ArmySTARRS and PRISMO

# Convert 0 to 1 to make gender uniform 
phenos_sub["armystarrs"]["Gender"] = 1
[get_frequency(df = x, col='Gender') for x in phenos_sub.values()]

[1    254
 Name: Gender, dtype: int64,
 1    346
 Name: Gender, dtype: int64,
 1    117
 Name: Gender, dtype: int64]

In [226]:
def replace_elements(df, col, new_elements, verbose = None):
    
    """
    Function to replace the elements in a column, e.g female:2, male:1 
    Parameters:
    df: data frame in which you want to replace
    col: name of the column in which you want to replace the elements
    new_elements: new elements to replace with 
    verbose: Print some information, default None 
    
    """
    df = df.copy(deep = True)
    x = df[col].value_counts().index
    if(len(x) != len(new_elements)):
        raise ValueError("Elements to replace must have the same length as new elements")
    
    d = {x[i]:new_elements[i] for i in range(len(new_elements))} # make dictionary
    
    if verbose is True:
        print("Categories :\n", x)
        print("Replacing :\n", d)
    
    df[col] = df[col].replace(d)
    
    return(df)

In [227]:
# replace gender in DNHS
# In original study, M = 2, F = 1
# But here in ML we will replace it to make it uniform with other studies
# dnhs_final = replace_elements(df = phenos_sub['dnhs'], col='Gender', 
#                        new_elements=[2,1], verbose=True)

In [228]:
# before 
# phenos_sub['dnhs']['Gender'].value_counts()

In [229]:
# After replacing
# dnhs_final["Gender"].value_counts()

In [230]:
# replace gender in GTP
# gtp_final = replace_elements(df = phenos_sub['gtp'], col='Gender', 
#                            new_elements=[2,1], verbose=True)

In [231]:
# Before 
# phenos_sub['gtp']['Gender'].value_counts()

In [232]:
# gtp_final['Gender'].value_counts()

In [233]:
# replace race in GTP
# gtp_final = replace_elements(df = gtp_final, col = "Race",
#                               new_elements=[2,1], verbose=True)

In [234]:
# gtp_final["Race"].value_counts()

In [235]:
# gtp_final

In [236]:
# MRS
phenos_sub['mrs']['Gender'].value_counts()

1    254
Name: Gender, dtype: int64

In [237]:
phenos_sub['armystarrs']['Gender'].value_counts()

1    346
Name: Gender, dtype: int64

In [238]:
phenos_sub['prismo']['Gender'].value_counts()

1    117
Name: Gender, dtype: int64

In [239]:
# combine phenotype data
final = pd.concat([phenos_sub['mrs'],
                   phenos_sub['armystarrs'],
                   phenos_sub['prismo']],
                  sort = False)

In [240]:
final.shape

(717, 26)

In [241]:
final

,Age,Avoidance,Basename,Bcell,Cd4T,Cd8T,Childhood_Mt,Comp.2,Comp.3,Ewas_Id,Gender,Hyperarousal,Intrusion,Mdd,Mono,Neu,Nk,Pts_Severity,Ptsdpm,Race,Smos,Study,Traumanum,Visit,Visitkey,Ptsdlife
77,20.00000,3.00000,201858500010_R04C01,0.06982,0.11557,0.10634,NaN,-0.02316,0.03448,1058_0,1,5.00000,2.00000,0.00000,0.05471,0.62616,0.02740,10.00000,0.00000,5,-14.42993,MRS,8.00000,0.00000,100119,NaN
76,21.00000,8.00000,201858500010_R03C01,0.03482,0.08983,0.09597,NaN,-0.02316,0.03448,1058_3,1,7.00000,3.00000,1.00000,0.05674,0.68559,0.03706,18.00000,0.00000,5,-14.66250,MRS,10.00000,3.00000,103964,NaN
63,21.00000,12.00000,201533590058_R02C01,0.02957,0.12690,0.12345,NaN,-0.03292,-0.01291,1148_0,1,10.00000,0.00000,11.00000,0.05332,0.65024,0.01651,22.00000,0.00000,5,18.71351,MRS,14.00000,0.00000,100208,NaN
62,22.00000,3.00000,201533590058_R01C01,0.02668,0.15131,0.11937,NaN,-0.03292,-0.01291,1148_2,1,19.00000,5.00000,4.00000,0.06432,0.60346,0.03486,27.00000,0.00000,5,21.32411,MRS,15.00000,2.00000,101866,NaN
245,21.00000,4.00000,202410280158_R08C01,0.00214,0.12064,0.09551,NaN,-0.08241,-0.00894,1340_0,1,23.00000,5.00000,9.00000,0.07120,0.66415,0.04637,32.00000,0.00000,5,-29.20186,MRS,3.00000,0.00000,100350,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,20.22253,NaN,201233760084_R01C01,0.03351,0.10032,0.10377,1.00000,-0.02648,0.06910,U.A.7.841,1,NaN,NaN,NaN,0.04434,0.61882,0.09923,NaN,NaN,oth,-16.01063,Prismo,NaN,0_epic,U.A.7.841.0,NaN
114,21.61125,9.00000,201233760084_R03C01,0.04360,0.12664,0.12946,1.00000,-0.02329,0.06724,U.A.7.841,1,10.00000,6.00000,NaN,0.05086,0.57904,0.07040,25.00000,0.00000,oth,-34.69551,Prismo,4.00000,2_epic,U.A.7.841.2,NaN
115,21.58184,18.00000,201233760084_R04C01,0.03957,0.15071,0.10001,6.00000,0.05563,0.03870,A.1.025,1,17.00000,10.00000,NaN,0.09667,0.57299,0.04004,45.00000,1.00000,eur,0.59052,Prismo,NaN,2_epic,A.1.025.2,NaN
116,21.00000,NaN,201233760084_R06C01,0.04812,0.18336,0.12530,6.00000,0.05416,0.03520,A.1.025,1,NaN,NaN,NaN,0.09192,0.50584,0.04546,NaN,NaN,eur,-2.26329,Prismo,NaN,0_epic,A.1.025.0,NaN


In [242]:
# Check categories in final
print("Gender:\n", final['Gender'].value_counts())
print("PTSDpm:\n", final['Ptsdpm'].value_counts())
print("PTSDlife:\n", final['Ptsdlife'].value_counts())

Gender:
 1    717
Name: Gender, dtype: int64
PTSDpm:
 0.00000    564
1.00000    128
Name: Ptsdpm, dtype: int64
PTSDlife:
 0.00000    285
1.00000     61
Name: Ptsdlife, dtype: int64


In [243]:
# now check na in the combined data
final.isnull().sum()

Age               1
Avoidance       204
Basename          0
Bcell             0
Cd4T              0
Cd8T              0
Childhood_Mt     40
Comp.2            0
Comp.3            0
Ewas_Id           0
Gender            0
Hyperarousal    203
Intrusion       202
Mdd             463
Mono              0
Neu               0
Nk                0
Pts_Severity    201
Ptsdpm           25
Race             10
Smos              0
Study             0
Traumanum        67
Visit             0
Visitkey          0
Ptsdlife        371
dtype: int64

In [244]:
# Now check if we have all the samples in pheno and methylation files
dfs_merged.columns.str.contains('|'.join(final['Basename'].tolist())).sum()

717

In [245]:
def check_all_match(first, second):
    """
    Function to check if all the samples in methylation and phenotye match
    Parameters: 
    first: Elements to search
    second: Elements to search in
    """
    all_match = first.str.contains('|'.join(second.tolist())).all()
    num_match = first.str.contains('|'.join(second.tolist())).sum()
    if not all_match:
        raise ValueError('All are not matching')
    elif all_match:
        print("All samples match between pheno and methylation: ", num_match)


In [246]:
check_all_match(first = final['Basename'], 
               second = dfs_merged.columns)

All samples match between pheno and methylation:  717


In [247]:
# Now save the data
# Create the directory and assign timestamp folder

import os, datetime

def make_directory(maindir = None, verbose = None):
    """
    Function to create directory in you current working directory.
    The function will have time stamp assigned
    
    Parameters: 
    dirname : name of main directory to hold newly created directories
    
    """
    
#     os.chdir('..') # go one step back to the current dir
    
    if maindir is False or  maindir is True:
        raise ValueError("dirname can't be True or False")
    
    if maindir is None:
        mydir = os.path.join(os.getcwd(),
                     datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
        
    elif maindir is not None:
        mydir = os.path.join(os.getcwd(), maindir,
                     datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S'))
        
    os.makedirs(mydir)
        
    if verbose:
        print("Directory created:", mydir)
        
    return(mydir)
      


In [248]:
# change to directory and make folder
# os.chdir("G:/PGC ML/")
# mydir = make_directory(maindir="Pre_Processed Data",  verbose=True)

# use directory that is already created
mydir = "G:/PGC ML/Pre_Processed Data/2021-11-15_21-41-53"

In [249]:
# Save Phenotype file
final.to_csv(os.path.join(mydir, "Pre_Post_MRS_ArmyS_Prismo_Pheno.csv"),
            index=False)

In [250]:

def save_data(fname, df):
    """
    Function to save the data
    Parameters:
    fname: file name
    df: data frame
    """
    if fname.endswith(".csv"):
        df.to_csv(os.path.join(mydir, fname))
    elif fname.endswith(".feather"):
        feather.write_feather(df, os.path.join(mydir, fname))
        

In [251]:
# Save individual datasets
# pheno_f_names = ["DNHS_UnqRESP_Pheno_final.csv", "GTP_Pheno_final.csv",
#           "MRS_POST_DEP_Pheno_final.csv", "ArmyStarrs_visit2_pheno.csv",
#           "Prismo_visit2_pheno.csv"]

# individual_cohorts = [dnhs_final, gtp_final, phenos_sub['mrs'],
#                   phenos_sub['armystarrs'], phenos_sub['prismo']]

In [252]:
# for i in range(len(pheno_f_names)):
#     save_data(fname=pheno_f_names[i], df = individual_cohorts[i])
#     print(pheno_f_names[i])
    

In [253]:
# Save individual methylation data 
# meth_f_names = ["DNHS_methylation_unq.feather", "GTP_methylation.feather",
#                "MRS_methylation_post.feather",
#                 "ArmyStarrs_visit2_methylation.feather",
#                "Prismo_visit2_methylation.feather"]
# for i in range(len(meth_f_names)):
#     save_data(fname=meth_f_names[i], df = all_meth_dfs[i])
#     print(meth_f_names[i])

In [257]:
# Save combined methylation data
feather.write_feather(dfs_merged, os.path.join(mydir, "Pre_Post_ArmyS_Prismo_methylation.feather"))

In [258]:
# Total number of columns that are matching 
# Without rowname column
dfs_merged.columns.isin(final["Basename"]).sum()

717

In [ ]:
# end